In [1]:
import os
import dspy
import mlflow
import pandas as pd
from dotenv import load_dotenv
import kagglehub
import ujson

/Users/viktorsvan/Python_koder/ingenjor_A_och_maskininlarnings_kod/LLM-DSPy-project/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
lm = dspy.LM("gpt-4o", api_key=openai_api_key)

In [3]:
local_lm = dspy.LM("ollama_chat/llama3.2", api_base="http://localhost:11434", api_key="")
dspy.configure(lm=local_lm)

In [4]:
class Chat(dspy.Signature):
    "You are a helpful assistant that answer questions about cats and dogs."
    question: str = dspy.InputField(desc="Questions asked by the user")
    response: str = dspy.OutputField(desc="Response to the question")

class Model(dspy.Module):
    def __init__(self):
        super().__init__()
        self.respond = dspy.ChainOfThought(Chat)

    def forward(self, question: str):
        return self.respond(question=question)

In [5]:
model = Model()

In [6]:
#mlflow ui --port 5000
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("DSPy")
mlflow.dspy.autolog()

Laddar ner dataset med beskrivning: "Contains 583 unique questions and answers related to dogs and cats.
It includes questions related to health, training, lifestyle, breeds and commonly asked questions."

In [7]:
path = '/Users/viktorsvan/.cache/kagglehub/datasets/bishnushahi/dog-cat-qa/versions/2'
csv_path = os.path.join(path, 'Dog-Cat-QA.csv')

In [8]:
df = pd.read_csv(csv_path)
print(df.head())

   Unnamed: 0                                           Question  \
0           0  What are some examples of breed-typical person...   
1           1  How do working and herding dogs typically behave?   
2           2   Which breed of dogs is known for strong loyalty?   
3           3  What instincts do guarding dogs typically disp...   
4           4  How does breed specificity impact a dog's abil...   

                                              Answer  
0  Breed-typical personalities in dogs are develo...  
1  Working and herding dogs have business-like di...  
2  Collies and Akitas are known for their strong ...  
3  Guarding dogs tend to be protective of their t...  
4  Breed specificity affects how well dogs adapt ...  


In [9]:
df = df.drop('Unnamed: 0', axis=1)
print(df.head())

                                            Question  \
0  What are some examples of breed-typical person...   
1  How do working and herding dogs typically behave?   
2   Which breed of dogs is known for strong loyalty?   
3  What instincts do guarding dogs typically disp...   
4  How does breed specificity impact a dog's abil...   

                                              Answer  
0  Breed-typical personalities in dogs are develo...  
1  Working and herding dogs have business-like di...  
2  Collies and Akitas are known for their strong ...  
3  Guarding dogs tend to be protective of their t...  
4  Breed specificity affects how well dogs adapt ...  


In [18]:
df.columns = ['question', 'response']

Gör om det till dictionary par:

In [19]:
data = df.to_dict(orient='records')
data = [dspy.Example(**d).with_inputs('question') for d in data]

In [20]:
example = data[2]
example

Example({'question': 'Which breed of dogs is known for strong loyalty?', 'response': 'Collies and Akitas are known for their strong sense of loyalty.'}) (input_keys={'question'})

Till skillnad från vanlig ML träning: Så ska man dela upp datasetet med 20 procent till träning och 80 procent till validering. 

"For prompt optimizers in particular, it's often better to pass more validation than training.
20% training and 80% validation.

In [21]:
print(len(data))

583


In [22]:
import random

random.Random(0).shuffle(data) 

trainset = data[:116]   # 60% 
valset   = data[116:466]  # 20% 
testset  = data[466:583]     # 20% 

len(trainset), len(valset), len(testset)

(116, 350, 117)

Nu laddar jag in en Evaluation metric som i sig också är en DSPy module: 

In [23]:
from dspy.evaluate import SemanticF1

In [25]:
metric = SemanticF1(decompositional=True)

# Produce a prediction from our `cot` module, using the `example` above as input.
pred = model(**example.inputs())

# Compute the metric score for the prediction.
score = metric(example, pred)

print(f"Question: \t {example.question}\n")
print(f"Gold Response: \t {example.response}\n")
print(f"Predicted Response: \t {pred.response}\n")
print(f"Semantic F1 Score: {score:.2f}")

Question: 	 Which breed of dogs is known for strong loyalty?

Gold Response: 	 Collies and Akitas are known for their strong sense of loyalty.

Predicted Response: 	 Some specific breeds that are known for strong loyalty include German Shepherds, Doberman Pinschers, and Rottweilers. These breeds have been bred for centuries to be protective of their families and territories, making them highly devoted companions.

Semantic F1 Score: 0.86


[Trace(request_id=b706d6c49d244b2c8c3a2a662e46e51a), Trace(request_id=9eaefcaad9d04f01ba599f2de8657cae)]